In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
import pickle
from sklearn.cluster import DBSCAN

In [2]:
df = pd.read_csv("../interaction_filtered.csv")

In [3]:
columns_to_drop = [
    "user_id", "click", "mod_price", "root_id",
    "category_id", "exposed_time" , "p_hour", "p_date" , "author_id" , "parent_id","title"
]

In [4]:
for i in columns_to_drop:
    df = df.drop(columns=i)

In [5]:
df.head()

,pid,category_level,author_fans_count,watch_time,duration,cvm_like,comment,follow,collect,forward,hate,tag_name,gender,age,fre_city,fre_community_type,fre_city_level
0,97474,1,46761,136,91.9,False,False,False,False,False,False,正能量,M,42,邯郸,unknown,三线城市
1,97474,1,46761,136,91.9,False,False,False,False,False,False,电视机,M,42,邯郸,unknown,三线城市
2,97474,1,46761,136,91.9,False,False,False,False,False,False,内容过于真实,M,42,邯郸,unknown,三线城市
3,97474,2,46761,136,91.9,False,False,False,False,False,False,正能量,M,42,邯郸,unknown,三线城市
4,97474,2,46761,136,91.9,False,False,False,False,False,False,电视机,M,42,邯郸,unknown,三线城市


In [13]:
import sys
sys.path.append('../')  # Adjust the path based on where internal_logics folder is located

from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from internal_logics.freq_mapper import SimplifiedFrequencyMapper
from internal_logics.label_encode import SimplifiedLabelEncoder
from internal_logics.gender_transform import GenderTransformer
from internal_logics.log_transform import LogTransformer
from internal_logics.gender_transform import GenderTransformer as GenderTransformer
from internal_logics.log_transform import LogTransformer as LogTransformer                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

boolean_cols = ["cvm_like", "comment", "follow", "collect", "forward", "hate"]
categorical_cols = ["tag_name", "fre_community_type", "fre_city_level", "fre_city"]
skewed_numeric_cols = ["watch_time", "duration", "author_fans_count"]

skewed_numeric_pipeline = Pipeline(steps=[
    ('log', LogTransformer()),
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ("booleans", SimplifiedLabelEncoder(), boolean_cols),
        ("categoricals", Pipeline(steps=[
            ('freq_map', SimplifiedFrequencyMapper()),
            ('label_enc', SimplifiedLabelEncoder())
        ]), categorical_cols),
        ("gender", GenderTransformer(), ["gender"]),
        ("skewed_numeric", skewed_numeric_pipeline, skewed_numeric_cols)
    ],
    remainder='passthrough'
)

In [14]:
full_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('pca', PCA(n_components=0.95))
])

In [15]:
sample_of_data = df.sample(500000)

In [16]:
# sample_only_preprocessor = preprocessor.fit_transform(sample_of_data)

In [17]:
# sample_only_preprocessor.shape

In [18]:
pca = PCA(n_components=0.95)

In [19]:
# sample_both_preProcessor_PCA = pca.fit_transform(sample_only_preprocessor)

In [22]:
# np.save("transformed_data_for_dbscan_testing.npy", sample_both_preProcessor_PCA)

In [29]:
transformed_data = np.load("transformed_data_for_dbscan_testing.npy")

In [25]:
# transformed_data.shape

In [26]:
# here hte transformed_data is the  numpy array , not the dataframe .

In [31]:
from sklearn.cluster import DBSCAN
import pandas as pd
from sklearn.preprocessing import StandardScaler

df = pd.DataFrame(transformed_data)

# Fit and predict with DBSCAN
dbscan = DBSCAN(eps=5, min_samples=15)
cluster_labels = dbscan.fit_predict(df)
print(cluster_labels)
with open("cluster_labels.pkl", "wb") as f:
    pickle.dump(cluster_labels, f)

# Add results to original DataFrame
df["dbscan_cluster_label"] = cluster_labels

# Print the number of clusters (excluding -1 which is noise/outliers)
n_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)

print(f"Number of clusters found without noise: {n_clusters}")
print(f"total number of clusters : {len(set(cluster_labels))}")



[   0    1    2 ... 1797  410 2225]
Number of clusters found without noise: 3443
total number of clusters : 3444


Now , after we have the clusters , we want to predict with new data points , for that , we will take new data of the user , then go the position of it in the same space , then we have to go for the closest cluster . But how to calculate which is hte closest cluster , because the DBSCAN has no predict method , we have to do that ourselves . By common sense , we have the calculate which is the closest cluster to user datapoint . which distance can be calculated by the center of cluster.
So we have to calculate the centeroids of the clusters:

In [30]:
# This code should run after you have trained your main DBSCAN model
# (after the cell with 'dbscan = DBSCAN(eps=5, min_samples=15)')

# 1. Filter out the noise points (label == -1) before calculating centroids
core_data = df[df['dbscan_cluster_label'] != -1]

# 2. Group by cluster label and calculate the mean of all features
# This gives us our "pseudo-centroids"
cluster_centroids = core_data.groupby('dbscan_cluster_label').mean()

# 3. Save the centroids to a file for later use in prediction
import pickle

with open("cluster_centroids.pkl", "wb") as f:
    pickle.dump(cluster_centroids, f)

print("Cluster centroids calculated and saved successfully.")
print(f"Found {len(cluster_centroids)} centroids.")

Cluster centroids calculated and saved successfully.
Found 3443 centroids.


In [ ]:
#creating the dataframe with the pid ,and one df with labels
pid = pd.DataFrame(df['pid'])
dbscan_cluster_label = pd.DataFrame(cluster_labels)

# 1. Combine the 'pid' and 'dbscan_cluster_label' side-by-side (axis=1)
df_video_clusters = pd.concat([pid, dbscan_cluster_label], axis=1)

# 2. Assign the correct column names (if they aren't already set)
df_video_clusters.columns = ['pid', 'dbscan_cluster_label']

# 3. Save to CSV
df_video_clusters.to_csv("video_clusters.csv", index=False)

# 4. View the result
print("File saved successfully!")
df_video_clusters.head()

calculating how many data points are there with each cluster.
there were total of 500k data points in the sample , 
and there are only 3444 clusters

In [ ]:
for i in range (0,3444):
    cluster_points = df[df["dbscan_cluster_label"] == i]
    print(f"The the datapoints that belong to cluster label{i} are")
    print(len(cluster_points))

In [40]:
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score

In [1]:
from sklearn.metrics import silhouette_score

mask = df["dbscan_cluster_label"] != -1
if mask.sum() > 1 and len(set(df['dbscan_cluster_label'][mask])) > 1:
    # 7. Evaluate the model on the non-noise data points
    sil_score = silhouette_score(df[mask], df['dbscan_cluster_label'][mask])
    db_score = davies_bouldin_score(df[mask], df['dbscan_cluster_label'][mask])
    ch_score = calinski_harabasz_score(df[mask], df['dbscan_cluster_label'][mask])

    print("--- DBSCAN Evaluation ---")
    print(f"Silhouette Score: {sil_score} (higher is better)")
    print(f"Davies-Bouldin Index: {db_score} (lower is better)")
    print(f"Calinski-Harabasz Score: {ch_score} (higher is better)")
    print("-------------------------")
else:
    print("Not enough clusters or data points for a meaningful evaluation. Adjust hyperparameters.")



NameError: name 'df' is not defined